# AI Agent for Market Research & Competitive Analysis

This application serves as an interactive demo for a sophisticated AI research agent. Simply enter a company name and stock ticker to generate a multi-faceted analysis from two distinct investor perspectives.

## Step 1: Install Dependencies & Setup Environment

This cell installs the required libraries, clones the project repository from GitHub to make the custom modules available, and sets up the necessary API keys for the agent to function.

In [ ]:
import importlib
import os
import sys
from google.colab import userdata

# Install dependencies from requirements.txt
if not importlib.util.find_spec("langchain"):
  print("Installing dependencies...")
  !pip install -q -r https://raw.githubusercontent.com/eriktaylor/ai-agent-moat/main/requirements.txt
else:
  print("Dependencies are already installed.")

# <<< CHANGE: Clone the repository and add it to the Python path >>>
repo_path = 'ai-agent-moat'
if not os.path.exists(repo_path):
    print(f"Cloning repository...")
    !git clone https://github.com/eriktaylor/ai-agent-moat.git
else:
    print("Repository already cloned.")

if repo_path not in sys.path:
    sys.path.append(repo_path)
    print(f"Added {repo_path} to system path.")

# Securely set up API keys for the agent's tools to use
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_CSE_ID'] = userdata.get('GOOGLE_CSE_ID')

## Step 2: Import and Initialize the Agent

Now we import our custom-built agent and tools. The heavy lifting and complex logic are handled in the background by our `research_agent.py` and `tools.py` files.

In [ ]:
# <<< CHANGE: Import the class, then initialize the models and agent here >>>
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from research_agent import ResearchAgent
from tools import get_stock_info

# Initialize models
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Instantiate the agent
research_agent = ResearchAgent(llm=llm, embeddings_model=embeddings)

print("AI Research Agent is initialized and ready.")

## Step 3: Run Your Analysis

Enter a company name and its corresponding stock ticker below to begin the analysis.

In [ ]:
from IPython.display import display, HTML, clear_output

company_name = input("Enter the company name (e.g., NVIDIA): ")
stock_ticker = input("Enter the stock ticker (e.g., NVDA): ")

clear_output(wait=True) # Clears the input prompts for a cleaner display

# To get fresh data and not use the cache, you can uncomment the next line:
# research_agent.clear_cache()

# --- 1. KEY FINANCIAL DATA ---
print(f"--- 1. KEY FINANCIAL DATA for {stock_ticker.upper()} ---")
financial_data_raw = get_stock_info.run(stock_ticker) if stock_ticker else "No ticker provided."
display(HTML(f"<div style='border: 1px solid #444; border-radius: 8px; padding: 20px; white-space: pre-wrap; font-family: monospace; line-height: 1.6; background-color: #2c2c2e; color: #f0f0f0;'>{financial_data_raw}</div>"))

# --- 2. AI-GENERATED MARKET INVESTOR OUTLOOK ---
print(f"\n--- 2. AI-GENERATED MARKET INVESTOR OUTLOOK for {company_name} ---")
market_outlook = research_agent.generate_market_outlook(company_name, stock_ticker)
display(HTML(f"<div style='border: 1px solid #444; border-radius: 8px; padding: 20px; max-height: 500px; overflow-y: auto; white-space: pre-wrap; font-family: \"SF Pro Text\", \"Inter\", sans-serif; line-height: 1.6; background-color: #2c2c2e; color: #f0f0f0;'>{market_outlook}</div>"))

# --- 3. AI-GENERATED VALUE INVESTOR ANALYSIS ---
print(f"\n--- 3. AI-GENERATED VALUE INVESTOR ANALYSIS for {company_name} ---")
value_analysis = research_agent.generate_value_analysis(company_name, stock_ticker)
display(HTML(f"<div style='border: 1px solid #444; border-radius: 8px; padding: 20px; max-height: 500px; overflow-y: auto; white-space: pre-wrap; font-family: \"SF Pro Text\", \"Inter\", sans-serif; line-height: 1.6; background-color: #2c2c2e; color: #f0f0f0;'>{value_analysis}</div>"))